In [ ]:
#Here we are creating a maodel which will predict whether the comment is sarcastic or non-sarcastic. I have taken this dataset because even
#though it is binary classification, detecting sarcastic comments can be tough for a model created from scratch due to chances of overfitting 
#as the model has to find the underlying relationship between each comment and classify the phrases as well as the words which might sound
#sarcastic to a person. So this could be done with proper tokenization and with hyperparamters tuned to avoid overfitting and hence we 
#have used transformers which gives an API for pretrained models.We have used the pretrained BERT Sequence Classification model 
#in this case to classify the text as Sarcastic or Non-Sarcastic

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! pip install transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from transformers import pipeline
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Spam Classifier/Data.csv')

In [ ]:

df

,headlines,target
0,CNN Triumphs (At Least in Most Demographic Cat...,Non Sarcastic
1,"‘You Did The Best You Could,’ Says Iron Man Ac...",Sarcastic
2,New Emails Reveal Warm Relationship Between Ka...,Non Sarcastic
3,Donald Trump Jr. Gets Slammed Over Racist Birt...,Non Sarcastic
4,God Urges Rick Perry Not To Run For President,Sarcastic
...,...,...
12501,Lapdogs of the Corporate Press,Non Sarcastic
12502,Annoying Coworker Precedes All Nouns With 'Qui...,Sarcastic
12503,Report: North Dakota Leads Nation In Parking A...,Sarcastic
12504,The Election Night Drinking Game: HyperVocal E...,Non Sarcastic


In [ ]:
count = 0
for i in   list(df['headlines']==None):
  if i==True:
    count = count+1

In [ ]:
X = list(df['headlines'])
y = list(df['target'])

In [ ]:
y = list(pd.get_dummies(y,drop_first=True)['Sarcastic'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=False, padding=True)
test_encodings = tokenizer(X_test, truncation=False, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=2,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=16,
    eval_steps = 10,   
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=10,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,       
    eval_dataset=test_dataset             
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/content/drive/MyDrive/Spam Classifier/model')

In [ ]:
from transformers import TFDistilBertForSequenceClassification, AutoModel
model = TFDistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Spam Classifier/model')

Some layers from the model checkpoint at /content/drive/MyDrive/Spam Classifier/model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Spam Classifier/model and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = TFTrainer(model, args = training_args)

/usr/local/lib/python3.9/dist-packages/transformers/trainer_tf.py:118: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.05975394643795718}

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm=confusion_matrix(y_test,output)
cm

array([[1323,    0],
       [   0, 1179]])

In [ ]:
accuracy_score(y_test, output)

1.0